In [70]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/breast-cancer/data.csv


In [71]:
data = pd.read_csv('../input/breast-cancer/data.csv')

In [72]:
##depending on the graphs, these features have more significant differences
#features_selection = ['radius_mean', 'perimeter_mean', 'area_mean', 'concavity_mean', 'concave points_mean']

In [73]:
## ML Packages

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

import time

In [74]:
features_selection=['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean','smoothness_mean','compactness_mean','concavity_mean','concave points_mean','symmetry_mean','fractal_dimension_mean','radius_se','texture_se','perimeter_se','area_se','smoothness_se','compactness_se','concavity_se','concave points_se','symmetry_se','fractal_dimension_se','radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst','concave points_worst','symmetry_worst','fractal_dimension_worst']


data_1=data.drop([data.index[568]])
print(data_1.shape)



X = data_1.loc[:,features_selection]
y = data_1.loc[:, 'diagnosis']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5, random_state = 49)

accuracy_all = []
cvs_all = []


(568, 33)


In [75]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential # initialize neural network library
from keras.layers import Dense # build our layers library
def build_classifier():
    classifier = Sequential() # initialize neural network
    classifier.add(Dense(units = 100, kernel_initializer = 'uniform', activation = 'relu', input_dim = X_train.shape[1]))
    classifier.add(Dense(units = 500, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, epochs = 200)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 3)
mean = accuracies.mean()
variance = accuracies.std()
print("Accuracy mean: "+ str(mean))
print("Accuracy variance: "+ str(variance))



from sklearn.model_selection import GridSearchCV, KFold
from tensorflow.keras.optimizers import Adam
# Define the grid search parameters
batch_size = [10,20,40]
epochs = [10,50,100]
# Make a dictionary of the grid search parameters
param_grid = dict(batch_size = batch_size,epochs = epochs)
# Build and fit the GridSearchCV
grid = GridSearchCV(estimator = classifier,param_grid = param_grid,cv = KFold(),verbose= 10)
grid_result = grid.fit(X,y)
print('Best: {} using {}'.format(grid_result.best_score_,grid_result.best_params_))

means=grid_result.cv_results_['mean_test_score']

stds=grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{},{} with: {}' .format(mean,stdev,param))




Epoch 1/200
6/6 [==============================] - 1s 4ms/step - loss: 2.4759 - accuracy: 0.4815
Epoch 2/200
6/6 [==============================] - 0s 3ms/step - loss: 1.5089 - accuracy: 0.5661
Epoch 3/200
6/6 [==============================] - 0s 3ms/step - loss: 0.9839 - accuracy: 0.5238
Epoch 4/200
6/6 [==============================] - 0s 3ms/step - loss: 0.6943 - accuracy: 0.6667
Epoch 5/200
6/6 [==============================] - 0s 3ms/step - loss: 0.5223 - accuracy: 0.7989
Epoch 6/200
6/6 [==============================] - 0s 3ms/step - loss: 0.3874 - accuracy: 0.8836
Epoch 7/200
6/6 [==============================] - 0s 3ms/step - loss: 0.3577 - accuracy: 0.9101
Epoch 8/200
6/6 [==============================] - 0s 3ms/step - loss: 0.2837 - accuracy: 0.9101
Epoch 9/200
6/6 [==============================] - 0s 3ms/step - loss: 0.2802 - accuracy: 0.9206
Epoch 10/200
6/6 [==============================] - 0s 3ms/step - loss: 0.2690 - accuracy: 0.9153
Epoch 11/200
6/6 [===========

In [76]:
import sklearn
value_1=sklearn.metrics.f1_score(y_train, y_test, average='binary', pos_label='M')
print(value_1)




0.43396226415094336
